In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

In [78]:
df= pd.read_csv("data/training2.csv")
na_cols=['mvar1', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar34', 'mvar35', 'mvar36', 'mvar37', 'mvar38',
         'mvar39', 'mvar43', 'mvar45', 'mvar46']

missing_cols= ['mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13', 'mvar15', 'mvar25',
               'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar40', 'mvar41', 'mvar42']

null_cols= list(df.columns[df.isnull().sum() >0])

#non-missing cols == ['mvar47', 'mvar14', 'default_ind', 'application_ind']
df.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,1.0,73.780,82.547,0.08696,10.0,0.63899,NaN,0.0,0,0
1,230033,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,...,0.0,99.129,NaN,0.00000,13.0,0.63836,NaN,NaN,1,1
2,230034,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,...,0.0,NaN,29.290,0.00000,1.0,1.00000,NaN,0.0,0,1
3,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,0.0,96.272,NaN,0.15385,3.0,0.53241,0.0,0.0,1,0
4,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,0.0,115.019,NaN,0.00000,1.0,0.92665,NaN,NaN,1,0


In [79]:
df[missing_cols]= df[missing_cols].fillna(0)
df= df.fillna(df.median())
y= df['default_ind']
cols= ['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6','mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20','mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25',
       'mvar26', 'mvar27','mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34','mvar35', 'mvar36', 'mvar37',
       'mvar38', 'mvar39', 'mvar40', 'mvar41','mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']
drop= ['mvar4', 'mvar5', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar34','mvar35', 'mvar39', 'mvar40', 'mvar41',
       'mvar45', 'mvar46']
X= df[cols]
#X= X.drop(drop, axis=1)
le= LabelEncoder()

In [73]:
test= pd.read_csv("data/Leaderboard_dataset.csv")
test= test.replace('missing', np.nan)
test= test.replace('na', np.nan)
test.isnull().sum()

test[na_cols]= test[na_cols].astype(str).astype(float)
test[missing_cols]= test[missing_cols].astype(str).astype(float)

test['mvar47']= le.fit_transform(test['mvar47'])
test[missing_cols]= test[missing_cols].fillna(0)
test= test.fillna(test.median())
test.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,350053,1753.0,0.5001,0.000,0.0,0.0,206.0,19179.0,206.0,14221.0,...,1.0,0.0,0.000,20.551,0.42105,3.0,0.85661,0.0,0.0,1
1,350054,1894.0,1.9701,0.000,0.0,0.0,297.0,19820.0,19820.0,176895.0,...,3.0,0.0,0.000,63.047,0.00000,15.0,0.94391,0.0,0.0,0
2,350055,1774.0,0.1718,0.000,0.0,0.0,197.0,2563.0,877.0,8869.0,...,1.0,0.0,0.000,83.797,0.33333,3.0,0.76467,0.0,0.0,0
3,350056,1586.0,0.1123,5.299,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.000,0.000,1.50000,0.0,0.73274,0.0,0.0,0
4,350057,1832.0,1.4442,0.000,0.0,0.0,179.0,7577.0,179.0,21059.0,...,3.0,0.0,96.052,66.665,0.00000,23.0,0.79190,0.0,0.0,1


In [74]:
Xtest= test.drop(['application_key'], axis=1)
#Xtest= Xtest.drop(drop, axis=1)
ids= test['application_key']

In [80]:
from sklearn.preprocessing import StandardScaler
scl= StandardScaler()
sm = SMOTE(random_state=2)

#X, y= sm.fit_sample(X,y)

Xtest= scl.fit_transform(Xtest)
X= scl.fit_transform(X)

In [49]:
rf=RandomForestClassifier(n_estimators=200, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X, y)

predictions= rf.predict_proba(Xtest)

In [50]:
data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp9.csv", index=False)

In [52]:
test= pd.read_csv("data/Evaluation_dataset.csv")
test= test.replace('missing', np.nan)
test= test.replace('na', np.nan)
test.isnull().sum()

test[na_cols]= test[na_cols].astype(str).astype(float)
test[missing_cols]= test[missing_cols].astype(str).astype(float)

test['mvar47']= le.fit_transform(test['mvar47'])
test[missing_cols]= test[missing_cols].fillna(0)
test= test.fillna(test.median())


Xtest= test.drop(['application_key'], axis=1)
#Xtest= Xtest.drop(drop, axis=1)
ids= test['application_key']
Xtest= scl.fit_transform(Xtest)


In [81]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, shuffle=True, stratify= y)
len(X_train)

56000

In [82]:
rf=RandomForestClassifier(n_estimators=200, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.83      0.94      0.88     18091
          1       0.69      0.43      0.53      5909

avg / total       0.80      0.81      0.80     24000

0.812583333333


In [53]:
rf=RandomForestClassifier(n_estimators=500, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X, y)

predictions= rf.predict_proba(Xtest)

data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp9.csv", index=False)

# Trying Bagging from here

In [ ]:
model = BaggingClassifier(base_estimator=rf)
model.fit(X, y)
# It is the best model till now

In [59]:
predictions1= model.predict_proba(Xtest)

In [60]:
data= dict({'ids': ids, 'pred-0':predictions1[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp10.csv", index=False)

In [68]:
test= pd.read_csv("data/Evaluation_dataset.csv")
test= test.replace('missing', np.nan)
test= test.replace('na', np.nan)
test.isnull().sum()

test[na_cols]= test[na_cols].astype(str).astype(float)
test[missing_cols]= test[missing_cols].astype(str).astype(float)

test['mvar47']= le.fit_transform(test['mvar47'])
test[missing_cols]= test[missing_cols].fillna(0)
test= test.fillna(test.median())


Xtest= test.drop(['application_key'], axis=1)
#Xtest= Xtest.drop(drop, axis=1)
ids= test['application_key']
Xtest= scl.fit_transform(Xtest)


In [62]:
model = BaggingClassifier(base_estimator=rf)
model.fit(X, y)

In [69]:
predictions= model.predict_proba(Xtest)

data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp12.csv", index=False)

In [70]:
#Tune the bagging classifier above for leaderboard

In [ ]:
model = BaggingClassifier(base_estimator=rf, n_estimators= 100, max_features= 10)
model.fit(X, y)

predictions= model.predict_proba(Xtest)

data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp13.csv", index=False)